### Imports

In [7]:
# Run once in terminal (not inside notebook)
# pip install -r requirements.txt

import os
from dotenv import load_dotenv

# LangChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# LangGraph
from langgraph.graph import StateGraph, END

# CrewAI
from crewai import Agent, Task, Crew

### Load API keys & LLM

In [8]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1")
 
load_dotenv()

llm = ChatOpenAI(
    model="openai/gpt-oss-20b:free",
    temperature=0,
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_API_BASE,
)

### Langchain Explanation prompt

In [9]:
explain_prompt = PromptTemplate(
    input_variables=["case_data"],
    template="""
You are a government fraud detection analyst.

Analyze the following welfare beneficiary data and explain why the case is suspicious.

Case Data:
{case_data}

Provide:
- Bullet-point reasoning
- Mention duplicate identity, shared financial links, or abnormal registrations
- Keep explanation concise and professional
"""
)

explanation_chain = LLMChain(
    llm=llm,
    prompt=explain_prompt
)


### LangGraph reasoning workflow

In [10]:
def explain_node(state: dict) -> dict:
    explanation = explanation_chain.run(case_data=state["case"])
    return {"explanation": explanation}


workflow = StateGraph(dict)
workflow.add_node("explain_case", explain_node)
workflow.set_entry_point("explain_case")
workflow.add_edge("explain_case", END)

graph = workflow.compile()

### CrewAI multi-agent setup

In [11]:
fraud_analyst = Agent(
    role="Fraud Analyst",
    goal="Detect suspicious welfare fraud patterns",
    backstory="Expert in government welfare fraud analytics.",
)

auditor = Agent(
    role="Government Auditor",
    goal="Summarize fraud evidence clearly",
    backstory="Responsible for compliance and investigation reports.",
)

investigation_task = Task(
    description="""
Investigate the suspicious welfare case and produce a final explanation
based on duplicate Aadhaar, shared bank accounts, and abnormal registrations.
""",
    expected_output="A detailed explanation of why the case is suspicious, including key indicators.",
    agent=fraud_analyst,
)

audit_task = Task(
    description="""
Create a concise audit-ready explanation of why the case is suspicious.
""",
    expected_output="A concise summary of the fraud evidence for audit purposes.",
    agent=auditor,
)

crew = Crew(
    agents=[fraud_analyst, auditor],
    tasks=[investigation_task, audit_task],
    llm=llm,
    verbose=True,
)

### Run suspicious case example

In [12]:
suspicious_case = {
    "aadhaar_like_id": "4587 9214 6632",
    "phone_number": "9876543210",
    "bank_account": "123456789012",
    "registrations_per_aadhaar": 3,
    "bank_shared_count": 5,
    "phone_shared_count": 4,
    "agent_cluster_size": 18,
    "annual_income": "₹85,000",
    "district": "Bhopal"
}

# LangGraph reasoning
result = graph.invoke({"case": suspicious_case})

print("\n--- AI Explanation (LangGraph) ---\n")
print(result["explanation"])

# CrewAI investigation
print("\n--- CrewAI Investigation ---\n")
crew.kickoff()


--- AI Explanation (LangGraph) ---

**Suspicious indicators for the beneficiary**

- **Duplicate identity risk** – The Aadhaar‑like ID is linked to **3 separate registrations** in the welfare system, far above the normal threshold of 1–2 for a single individual.  
- **Shared financial links** –  
  - The bank account “123456789012” is shared by **5 different beneficiaries**.  
  - The phone number “9876543210” is shared by **4 beneficiaries**.  
  These high sharing counts strongly suggest a common “master” identity used to create multiple benefit claims.  
- **Agent cluster anomaly** – The beneficiary is part of an agent cluster of **18 people**. Clusters that large are often used to coordinate bulk registrations or to funnel benefits through a single point of contact.  
- **Income‑to‑benefit mismatch** – An annual income of ₹85,000 is well below the poverty threshold for the Bhopal district, yet the beneficiary has multiple registrations, raising the possibility of benefit stacking.

APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 16384 tokens, but can only afford 2673. To increase, visit https://openrouter.ai/settings/credits and upgrade to a paid account', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_2vQYafVdKik7UPzFQqK2y5KyyFp'}